In [75]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

np.random.seed( 235711 )

data = np.loadtxt('C:\\Git_NEWWWWWWWWW\\StructEst_W19\\ProblemSets\\PS4\\data\\NewMacroSeries.txt', delimiter = ',')
c_t, k_t, w_t, r_t, y_t = data[:,0], data[:,1], data[:,2], data[:,3], data[:,4]


In [76]:
#draw from uniform distribution - 1000 samples S (cols) of 100 data points T (rows)
sim = np.random.uniform(0,1,(100,1000))

In [77]:
#define k_1
k_1 = k_t.mean()

#define simulated variables
def simvar(alpha, beta, rho, mu, sigma, sim_var):
    eps = sts.norm.ppf(sim_var,0,sigma)
    z = np.zeros((100,1000))  # of data points (T=100) and # of samples (T=1000)
    z_t_1 = mu
    for i in range(100):   
        z_t = rho*z_t_1 + (1-rho)*mu + eps[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
    
    k = np.zeros((100,1000))
    k_lag = np.zeros((100,1000))
    k_t_1 = k_1
    for i in range(100):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_lag[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t

    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag-k
    y = np.exp(z)*k_lag**alpha
    
    return c,k_lag,k,w,r,y

In [78]:
simvar(0.5,0.99,0.5,8,0.5, sim)

(array([[ 5625809.39123465,  2676995.60037952,  3797915.0310855 , ...,
          3800952.31062879,  3796640.4511661 ,  3269322.83055408],
        [ 8863038.89618396,  2353867.00761104,  2778296.85880221, ...,
          5803696.59715247,  2999496.44148037,  2069867.79122435],
        [ 4484647.61121418,  2966434.18654657,  3641721.79909407, ...,
          2708592.87274338,  2475601.86259   ,  1504382.03438814],
        ...,
        [ 3432138.86884835,  1641416.15204297,  2659719.91880952, ...,
          5653355.04188805,   747865.48620359, 15598262.26748797],
        [ 1141998.45269034,  1038100.73225016,  2795640.80186753, ...,
          3561429.91905864,   591189.31557541,  6907727.68602137],
        [  424663.61056944,  1831388.19731965,  7011985.26161415, ...,
          1157663.48208465,  1233084.38488111,  4313541.20451569]]),
 array([[ 6643985.13829907,  6643985.13829907,  6643985.13829907, ...,
          6643985.13829907,  6643985.13829907,  6643985.13829907],
        [ 5514407.2

In [79]:
#model moments (mm)

def sim_mom(alpha, beta, rho, mu, sigma, sim_var):
    c,k_lag,k,w,r,y = simvar(alpha, beta, rho, mu, sigma, sim_var)
    mm1 = np.mean(c)
    mm2 = np.mean(k)
    mm3 = np.mean(c/y)
    mm4 = np.mean(np.var(y,axis=0))   #changd std into var here
    corr_c = np.zeros((1,1000))
    corr_ck = np.zeros((1,1000))
    
    for i in range(1000):    #need to perform it for the number of samples S, over all T for each sample
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    mm5 = corr_c.mean()
    mm6 = corr_ck.mean()
    sim_mom = np.array([mm1,mm2,mm3,mm4,mm5,mm6])
    return sim_mom

#test
#print(sim_mom(.5, .99, .5, 10, .5,sim))

#data moments (dm)

dm1 = c_t.mean()
dm2 = k_t.mean()
dm3 = (c_t/y_t).mean()
dm4 = y_t.var()
dm5= np.corrcoef(c_t[0:99],c_t[1:100])[0,1]
dm6= np.corrcoef(c_t, k_t)[0,1]
data_moms=np.array([dm1, dm2, dm3, dm4, dm5, dm6])


def err_vec(alpha, beta, rho, mu, sigma, sim):
    model_moms = sim_mom(alpha, beta, rho, mu, sigma, sim)
    return (model_moms - data_moms)/data_moms

def SMM_crit(params, *args):
    alpha, rho, mu, sigma = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sigma, sim)
    W = np.eye(6)
    return err @ W @ err.T

In [80]:
#SMM simulation
beta=0.99
params_init = np.array([0.5, 0.9, 9, 0.2])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,1.1))
result = opt.minimize(SMM_crit, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bounds)
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x
print(result)

print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sigma_smm)

print('Error vec =', err_vec(alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, sim))
print("Criterion function value =", SMM_crit(result.x, 0.99, sim))

      fun: 4.327714757378358e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00144366, -0.00045429, -0.00010606, -0.0010308 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 525
      nit: 82
   status: 0
  success: True
        x: array([0.42105693, 0.92567283, 9.93799591, 0.08786121])
alpha SMM =  0.42105693496904173
rho SMM =  0.9256728306145695
mu SMM =  9.937995905701035
sigma SMM =  0.08786121246740376
Error vec = [ 6.98934335e-04 -6.91506342e-04 -1.79110856e-03 -1.91923742e-05
  2.80913435e-04 -2.71429479e-04]
Criterion function value = 4.327714757378358e-06


In [81]:
#Jacobian
# number of parameters = 4  (K)
# number of moments = 6  (R)

import numpy.linalg as lin

def Jac_err2(data_vals, unif_vals, alpha, rho, mu, sigma, simple=False):

    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    Jac_err[:, 0] = \
        ((err_vec(alpha + h_alpha, 0.99, rho, mu, sigma, sim) -
          err_vec(alpha - h_alpha, 0.99, rho, mu, sigma, sim)) / (2 * h_alpha)).flatten()
    Jac_err[:, 1] = \
        ((err_vec(alpha, 0.99, rho + h_rho, mu, sigma, sim) -
          err_vec(alpha, 0.99, rho - h_rho, mu, sigma, sim)) / (2 * h_rho)).flatten()
    Jac_err[:, 2] = \
        ((err_vec(alpha, 0.99, rho, mu + h_mu, sigma, sim) -
          err_vec(alpha, 0.99, rho, mu - h_mu, sigma, sim)) / (2 * h_mu)).flatten()
    Jac_err[:, 3] = \
        ((err_vec(alpha, 0.99, rho, mu, sigma + h_sigma, sim) -
          err_vec(alpha, 0.99, rho, mu, sigma - h_sigma, sim)) / (2 * h_sigma)).flatten()
    
    return Jac_err

S = sim.shape[1]
d_err2 = Jac_err2(data, sim, alpha_smm, rho_smm, mu_smm, sigma_smm, False)
print(d_err2)
W = np.eye(6)
print(W)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W @ d_err2)
print(SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. rho_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. sigma_hat=', np.sqrt(SigHat2[3, 3]))


[[ 2.70626362e+01  7.07368515e-01  1.71634382e+00  1.39784623e+00]
 [ 3.10948621e+01  7.06385648e-01  1.71395901e+00  1.39590396e+00]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.13556980e+01  1.23240631e+01  3.45880430e+00  2.75973921e+01]
 [ 1.41778555e-01  4.41913685e-01  7.95545293e-04 -8.95476345e-02]
 [ 2.18853975e-01  4.47472006e-01  4.99766732e-03 -8.79134812e-02]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 8.95653406e-05 -2.61551732e-05 -1.50900948e-03  1.67995869e-06]
 [-2.61551732e-05  2.14612090e-03  3.18940624e-04 -9.39933444e-04]
 [-1.50900948e-03  3.18940624e-04  2.56428001e-02 -2.57335311e-06]
 [ 1.67995869e-06 -9.39933444e-04 -2.57335311e-06  4.17671199e-04]]
Std. err. alpha_hat= 0.009463896692142535
Std. err. rho_hat= 0.046326244162780375
Std. err. mu_hat= 0.16013369433690638
Std. err. sigma_hat= 0.020437005616054136


In [103]:
# B) TWO STEP WEIGHTING MATRIX

#data moments (dm)

dm1 = c_t.mean()
dm2 = k_t.mean()
dm3 = (c_t/y_t).mean()
dm4 = y_t.var()
dm5= np.corrcoef(c_t[0:99],c_t[1:100])[0,1]
dm6= np.corrcoef(c_t, k_t)[0,1]

def get_Err_mat(unif_vals, alpha, beta, rho, mu, sigma, simple=False):
  
    R = 6
    S = sim.shape[1]
    Err_mat = np.zeros((R, S))
    
    c,k_lag,k,w,r,y = simvar(alpha, beta, rho, mu, sigma, sim)
    
    print( np.mean(c,0))
    
    mm1 = np.mean(c,0)
    mm2 = np.mean(k,0)
    mm3 = np.mean(c/y,0)
    mm4 = np.mean(np.var(y,axis=0))   #changd std into var here
    mm4 = np.mean( [np.var(y[:,i],axis=0) for i in range(1000)])
    corr_c = np.zeros((1,1000))
    corr_ck = np.zeros((1,1000))
    
    for i in range(1000):    #need to perform it for the number of samples S, over all T for each sample
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    mm5 = corr_c.mean()
    mm6 = corr_ck.mean()
    sim_mom = np.array([mm1,mm2,mm3,mm4,mm5,mm6])
    sim1 = mm1
    sim2 = mm2
    sim3 = mm3
    sim4 = mm4
    sim5 = corr_c
    sim6 = corr_ck
    
    
    #sim1, sim2, sim3, sim4, sim5, sim6 = sim_mom(alpha, beta, rho, mu, sigma, sim)
    if simple:
        Err_mat[0, :] = sim1 - dm1
        Err_mat[1, :] = sim2 - dm2
        Err_mat[2, :] = sim3 - dm3
        Err_mat[3, :] = sim4 - dm4
        Err_mat[4, :] = sim5 - dm5
        Err_mat[5, :] = sim6 - dm6
    else:
        Err_mat[0, :] = (sim1 - dm1) / dm1
        Err_mat[1, :] = (sim2 - dm2) / dm2
        Err_mat[2, :] = (sim3 - dm3) / dm3
        Err_mat[3, :] = (sim4 - dm4) / dm4
        Err_mat[4, :] = (sim5 - dm5) / dm5
        Err_mat[5, :] = (sim6 - dm6) / dm6
        
    #print(sim1, sim2, sim3, sim4, sim5, sim6)
    #print(dm1,dm2,dm3,dm4, dm5,dm6)
    #print(Err_mat)
    return Err_mat


In [123]:
Err_mat = get_Err_mat(sim, alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, False)
#XXX = (1 / sim.shape[1]) * (Err_mat @ Err_mat.T)

#print( Err_mat[2,:])
print( sim.shape[1])
print(sim.shape)

XXX = (1 / sim.shape[1]) * sum(np.outer( Err_mat[:,i], Err_mat[:,i]) for i in range(1000))
print(XXX)
print( XXX.shape )

#print(XXX)
W_hat = lin.inv(XXX)
print(W_hat)

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

print(is_pos_def(W_hat))

[ 8457771.6528173   8446300.94496495 10423230.76691658 11391354.26029423
  8086527.77624815  8451745.52417982  9016402.97597741 11739433.27320004
  8788148.90190764  7953704.08501463  8418780.58117881 10161051.40953356
  8407747.023597   11945128.44678612  9546220.65927535  7221857.05703349
 14311938.84401163  7774817.88034412  6400401.23163669 10431754.04820387
  9239563.72353135 10503738.0616202   7254421.76455203  8277968.64158196
  5870111.08624993 10861250.2018548   8430480.44167226  7673709.59122861
 10528615.1330515   8815971.7390916  13016838.72061692  5811915.28532339
  8508510.98047658 10906627.97885427  9185433.72001146 10023987.55104871
  8665271.69186352  8843173.77916542  8172342.49564396  8457446.28220827
  8453624.17146788  8495697.41473848  9354628.98287258 12711560.33256864
  7204341.78640508  8295306.85193534  6716074.73826072  7470043.73461863
  8791681.99781274  8419163.74554414  6713208.46020819  7931907.21641847
  7812098.56900072 11189005.40194378 10283330.41713

In [124]:
def SMM_crit2(params, *args):
    alpha, rho, mu, sigma = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sigma, sim)
    return err @ W_hat @ err.T

#SMM simulation
beta=0.99
params_init = np.array([0.5, 0.9, 9, 0.2])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,1.1))
result = opt.minimize(SMM_crit2, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bounds)
alpha_smm2, rho_smm2, mu_smm2, sigma_smm2 = result.x
print(result)

print('alpha SMM_2step = ', alpha_smm2)
print('rho SMM_2step = ', rho_smm2)
print('mu SMM_2step = ', mu_smm2)
print('sigma SMM_2step = ', sigma_smm2)

d_err2 = Jac_err2(data, sim, alpha_smm2, rho_smm2, mu_smm2, sigma_smm2, False)
print(d_err2)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W_hat @ d_err2)
print(SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. rho_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. sigma_hat=', np.sqrt(SigHat2[3, 3]))


      fun: 2.994232720946073e+23
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.34217728e+16,  0.00000000e+00,  1.66526955e+21,  4.86025881e+20])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 190
      nit: 12
   status: 0
  success: True
        x: array([ 6.78844356e-01, -1.00000000e+00,  5.00000000e+00,  1.00000000e-10])
alpha SMM_2step =  0.6788443557070718
rho SMM_2step =  -0.9999999999
mu SMM_2step =  5.0
sigma SMM_2step =  1e-10
[[ 3.94369667e+00 -1.11022302e-12  2.74469549e-01  0.00000000e+00]
 [ 1.24678966e+01 -3.88578059e-12  7.85783449e-01  0.00000000e+00]
 [-1.69462513e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.81619137e-02 -0.00000000e+00 -2.74643757e-03  0.00000000e+00]
 [ 7.12244882e-02 -0.00000000e+00  4.33873433e-03  5.89805982e-03]
 [ 1.54463857e-01 -0.00000000e+00  9.05339931e-03  0.00000000e+00]]
[[ 1.12017938e-09  3.97057150e+03 -1.38440163e-12  1.14973096e-07]
 [ 3.97057150e+03  1.40740298e+16